In [1]:
!pip install --force-reinstall dist/alquimia_fair_forge-0.0.1.tar.gz -q


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from fair_forge.metrics import Humanity
from helpers.retriever import CustomRetriever

/Users/alexfiorenza/Documents/software_development/projects/alquimia/fair-forge/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Emotional entropy
Based on NRC emotional lexicon and Plutchik eight basic emotions

TODO:
- Implement Emotion matching
- Implement Language Style Matching, LSM
- Implement Agreeableness
- Implement Empathy, Empathic Concern

In [3]:
metrics = Humanity.run(CustomRetriever,verbose=True)

2025-05-19 12:42:33,203 - fair_forge - INFO - Loaded dataset with 1 batches
2025-05-19 12:42:33,204 - fair_forge - INFO - Starting to process dataset
2025-05-19 12:42:33,205 - fair_forge - INFO - Processing batch 1/1
2025-05-19 12:42:33,206 - fair_forge - INFO - Session ID: 123, Assistant ID: 456
2025-05-19 12:42:33,480 - fair_forge - DEBUG - Assistant distribution: {'Anger': 0.0, 'Anticipation': 0.3333333333333333, 'Disgust': 0.0, 'Fear': 0.0, 'Joy': 0.3333333333333333, 'Sadness': 0.0, 'Surprise': 0.0, 'Trust': 0.3333333333333333}
2025-05-19 12:42:33,480 - fair_forge - DEBUG - Generated vector: [0.0, 0.3333333333333333, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.3333333333333333]
2025-05-19 12:42:33,480 - fair_forge - DEBUG - Spearman value: 0
2025-05-19 12:42:33,481 - fair_forge - INFO - Completed processing all batches. Total metrics collected: 1


In [ ]:
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
dataset = os.environ.get("dataset", "asb")
humanity_index = f"{dataset}-humanity"

In [3]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
mapping_humanity = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "humanity_assistant_emotional_entropy": {"type": "float"},
      "humanity_ground_truth_spearman": {"type": "float"},
      "humanity_assistant_anger": {"type": "float"},
      "humanity_assistant_anticipation": {"type": "float"},
      "humanity_assistant_disgust": {"type": "float"},
      "humanity_assistant_fear": {"type": "float"},
      "humanity_assistant_joy": {"type": "float"},
      "humanity_assistant_sadness": {"type": "float"},
      "humanity_assistant_surprise": {"type": "float"},
      "humanity_assistant_trust": {"type": "float"},
      "assistant_id": {"type": "keyword"},
      "qa_id": {"type": "keyword"},
    }
  }
}
recreate_index(humanity_index, mapping_humanity)

In [ ]:
docs = []
for metric in metrics:
    docs.append({
            "_index": humanity_index,
            "_source": metric.model_dump()
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")